In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [25]:
# a = [1,2,3]
# b = [4,5,6]
# zipped_list = zip(a,b)
zipped_list=[(1, 4), (3, 5), (2, 6)]
zipped_list
unzipped_list=list(zip(*zipped_list))
unzipped_list
list(unzipped_list[0])
list(unzipped_list[1])

[(1, 4), (3, 5), (2, 6)]

[(1, 3, 2), (4, 5, 6)]

[1, 3, 2]

[4, 5, 6]

In [36]:
p=[(1, 4), (3, 5), (2, 6)]

p.sort()
p

p=list(zip(*p))
p

p=[list(p[0]),list(p[1])]
p

[(1, 4), (2, 6), (3, 5)]

[(1, 2, 3), (4, 6, 5)]

[[1, 2, 3], [4, 6, 5]]

In [178]:
import numpy as np
# from toolbox import *
import toolbox as tb
from skeleton import *
import pandas as pd
from gensim.models import FastText
%load_ext autoreload
%autoreload 2
import os
import tqdm
from microtc.utils import tweet_iterator
import datetime
from collections import Counter
import gc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [227]:
class Tokens():
    tokenized_pointer=None
    
    def tokenize(self,json_file,npy_file=None,replace=False):
        self.npy_file=str(os.path.splitext(json_file)[0]+".npy") if not npy_file else npy_file

        if not replace and os.path.isfile(self.npy_file):
            print(f"** Replace is off. {os.path.abspath(self.npy_file)} already exists, then load.")
        else:
            tokenized_docs=[]
            self.N=0
            print(f"** Processing {json_file} ...")
            tx = datetime.datetime.now()
            for idx, tw in enumerate(tweet_iterator(json_file)):
                tb.show_progress(1000, tx, idx)
                twTokens = tb.process_line(tw['text']) # Limpiando text.
#                 print(type(twTokens))
                tokenized_docs.append(twTokens)
            
#             print("max:",len(max(tokenized_docs)))
    
            self.N=idx+1
#             print(type(tokenized_docs),len(tokenized_docs))
#             print(tokenized_docs)
#             tokenized_docs
            maxLen=len(max(tokenized_docs))
            for i, doc in enumerate(tokenized_docs):
#                 print("Before:",tokenized_docs[i])
#                 print(doc, ['']*(maxLen - len(doc)))
#                 tokenized_docs[i] = doc + ['']*(maxLen - len(doc))
                tokenized_docs[i]=[doc[x] if x<len(doc) else '' for x in range(maxLen)]
#                 print("After:",tokenized_docs[i])
            
            np.save(self.npy_file,tokenized_docs)
            del(tokenized_docs)
            gc.collect()
            print(f"** Processed {self.N} lines. Saved to {os.path.abspath(self.npy_file)}.")
        
        self.pointer=np.load(self.npy_file, mmap_mode='r')
        return self
    
    def get(self,n):
        return [x for x in self.pointer[n,:] if x != '']

In [229]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 03:23:16.540113 :: 0.000166 - Processing item #0 
** Processed 10 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [230]:
tokens.get(0)

['nochecit']

In [ ]:
class Index():
#     vocab = None
    N = 0
    DF = Counter({})
    TF = []
    
#     DF_vec = None
#     IDF_vec = None

    def computePostingLists(self, filename, showProgressEach=None):
        print("** Processing " + filename + "...")
#         tx = datetime.datetime.now()
        for idx, tw in tqdm(enumerate(tweet_iterator(filename))):
#             tb.show_progress(showProgressEach,tx,idx)
            twTxtLst = tb.process_line(tw['text']) # Limpiando text.
            
            # Sumando frecuencias individiales.
            twCnt = Counter(twTxtLst)
            
            TF.append(twCnt)
            
            # Sumando frecuencias por documento (DF).
            self.DF.update(list(twCnt.keys()))

            # Contabilizando documentos.
            self.N += 1
            
#         words = self.DF.keys()
#         self.vocab = dict(zip(words, range(len(words))))
#         self.DF_vec = cnt2vec(self.DF, self.vocab)
#         self.IDF_vec = np.log2(self.N / (self.DF_vec + 1))
        
#         # Calculando TF-IDF y asignando a usuarios.
#         for userId in self.users:
#             TF_cnt = self.users[userId]
#             self.users[userId] = cnt2vec(TF_cnt, self.vocab) * self.IDF_vec
        postlists=defaultdict(list)

    def getX(self):
        return np.array([self.users[key] for key in self.users])
    
    def cluster(self, n_clusters=9):
        self.cluster = KMeans(n_clusters=n_clusters, n_jobs=4).fit(self.getX())
        return self.cluster

In [232]:
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
print(vars_.sort())

Total = 128084
None
